# 一、T检验在scipy中的应用

In [2]:
from scipy.stats import ttest_ind,levene# ttest_ind 表示t检验  Levene用于检查方差齐性
import pandas as pd

In [7]:
data=pd.read_csv('../breast_cancer_m.csv')
data_a=data[data['label']==0]
data_b=data[data['label']==1]
data_a

,A,B,C,D,E,F,G,H,I,J,...,V,W,X,Y,Z,AA,AB,AC,AD,label
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563,20.92,25.09,143.00,1347.0,0.10990,0.22360,0.31740,0.14740,0.2149,0.06879,...,29.41,179.10,1819.0,0.1407,0.4186,0.6599,0.2542,0.2929,0.09873,0
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,26.40,166.10,2027.0,0.1410,0.2113,0.4107,0.2216,0.2060,0.07115,0
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,38.25,155.00,1731.0,0.1166,0.1922,0.3215,0.1628,0.2572,0.06637,0
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,34.12,126.70,1124.0,0.1139,0.3094,0.3403,0.1418,0.2218,0.07820,0


In [6]:
x_a=data_a.iloc[:,:-1]
x_b=data_b.iloc[:,:-1]
y_a=data_a.iloc[:,:-1]
y_b=data_a.iloc[:,:-1]
x_a.shape,x_b.shape

((212, 30), (357, 30))

In [10]:
levene(x_a['A'],x_b['A'])#对A特征进行方差齐性检测  只有p>0.05才认为具有方差齐性  结表明不具备方差齐性

LeveneResult(statistic=90.47705934341127, pvalue=5.279775501703329e-20)

In [12]:
ttest_ind(x_a['A'],x_b['A'],equal_var=False) #因为不具备方差齐性，所以euqal_var=False采取welch's t-test
#p<0.05表明具有显著性统计学差异 即这个特征可以被保留（有用）

Ttest_indResult(statistic=22.208797758464527, pvalue=1.6844591259582747e-64)

In [15]:
# 前面是对一个特征A进行t检验，现在利用循坏对所有特征进行t检验，筛选出有用特征

colNameSel_t=[]
for colName in x_a.columns[:]:#遍历每个特征
    if levene(x_a[colName],x_b[colName])[1]>0.05:#首先进行方差齐性检验，符合要求进行t-test 检验 
        if ttest_ind(x_a[colName],x_b[colName])[1]<0.05:#t-test检验得出p<0.05 筛选出该特征
            colNameSel_t.append(colName)
    else:  # 不满足方差齐性，则进行welch's t-test检验
        # t-test检验得出p<0.05 筛选出该特征
        if ttest_ind(x_a[colName], x_b[colName], equal_var=False)[1] < 0.05:
            colNameSel_t.append(colName)
print(len(colNameSel_t))
print(colNameSel_t)

25
['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'M', 'N', 'P', 'Q', 'R', 'U', 'V', 'W', 'X', 'Y', 'Z', 'AA', 'AB', 'AC', 'AD']


# 二、曼-惠特尼U秩和检验在scipy中的应用


In [18]:
from scipy.stats import mannwhitneyu


In [19]:
colNameSel_mwu=[]
for colName in x_a.columns[:]:
    if mannwhitneyu(x_a[colName], x_b[colName])[1] < 0.05:  # 曼-惠特尼U秩和检验得出p<0.05 筛选出该特征
        colNameSel_mwu.append(colName)
print(len(colNameSel_mwu))
print(colNameSel_mwu)


27
['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'AA', 'AB', 'AC', 'AD']


## 比较两种检验的结果

In [21]:
# 曼-惠特尼U秩和检验具有显著性而t检验不具有显著性的特征
set(colNameSel_mwu).difference(colNameSel_t)

{'S', 'T'}

In [22]:
# t检验具有显著性而曼-惠特尼U秩和检验不具有显著性的特征
set(colNameSel_t).difference(colNameSel_mwu)


set()